In [27]:
from faqer.services.data.utils import get_text, get_lines, tokenize_text
doc = get_text()

In [29]:
from gensim.models.ldamulticore import LdaMulticore
from gensim.corpora.dictionary import Dictionary

In [32]:
lines = []
for line in get_lines():
    lines.append(tokenize_text(line))

In [34]:
common_dictionary = Dictionary(lines)

[2020-12-30 10:08:24,224] [INFO] adding document #0 to Dictionary(0 unique tokens: [])
[2020-12-30 10:08:24,354] [INFO] built Dictionary(8558 unique tokens: ['award', 'email', 'get', 'like', 'mailto']...) from 5609 documents (total 63861 corpus positions)


In [35]:
common_corpus = [common_dictionary.doc2bow(text) for text in lines]

In [60]:
lda = LdaMulticore(common_corpus, num_topics=5)

[2020-12-30 10:22:08,972] [WARNING] no word id mapping provided; initializing from corpus, assuming identity
[2020-12-30 10:22:08,992] [INFO] using symmetric alpha at 0.2
[2020-12-30 10:22:08,994] [INFO] using symmetric eta at 0.2
[2020-12-30 10:22:08,997] [INFO] using serial LDA version on this node
[2020-12-30 10:22:09,003] [INFO] running online LDA training, 5 topics, 1 passes over the supplied corpus of 5609 documents, updating every 14000 documents, evaluating every ~5609 documents, iterating 50x with a convergence threshold of 0.001000
[2020-12-30 10:22:09,004] [WARNING] too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
[2020-12-30 10:22:09,005] [INFO] training LDA model using 7 processes
[2020-12-30 10:22:09,065] [INFO] PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/5609, outstanding queue size 1
[2020-12-30 10:22:09,075] [INFO] PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/5609

In [61]:
other_corpus = common_dictionary.doc2bow(['в', 'доступ', 'систему'])
lda[other_corpus]
sorted(lda[other_corpus], key=lambda x: x[1], reverse=True)[0]

(1, 0.7265626)

In [62]:
from collections import defaultdict

topic_dict = defaultdict(list)

for line in lines:
    other_corpus = common_dictionary.doc2bow(line)
    key, prob = sorted(lda[other_corpus], key=lambda x: x[1], reverse=True)[0]
    topic_dict[key].append(line)
    

In [70]:
lines

[['would', 'like', 'get', 'email', 'password', 'reset', 'mailto', 'award'],
 ['hello',
  'new',
  'password',
  'sent',
  'slack',
  'please',
  'future',
  'create',
  'ticket',
  'access',
  'queue'],
 ['spacibo'],
 ['привет',
  'живет',
  'добавлен',
  'нов',
  'ключ',
  'udsp',
  'попаст',
  'мог',
  'билет',
  'создава'],
 ['привет', 'билет', 'access'],
 ['ключ', 'билет', 'присоединя', 'вид', 'файл'],
 ['ключ', 'прит', 'публичн', 'долж'],
 ['блин', 'дума', 'приватн', 'спасиб'],
 ['хвата', 'нескольк', 'последн', 'коммит', 'сравнен', 'стар', 'hg', 'реп'],
 ['https', 'amp', 'пятисот'],
 ['fixed'],
 ['привет', 'знает', 'веб', 'хук', 'слак', 'завед'],
 ['https'],
 ['спасиб'],
 ['драстут', 'как', 'переключен', 'hg', 'авторизовыва'],
 ['изменен', 'затронут', 'browser', 'клиент', 'работа', 'работа'],
 ['кажет', 'завал', 'rt'],
 ['врод', 'работа'],
 [],
 ['ок', 'главн'],
 ['попроб', 'инкогнит', 'браузер'],
 ['заработа', 'главн', 'попроб', 'https', 'заработа'],
 ['no', 'permission', 'view',

In [71]:
from faqer.services.classificator.eval import RDTModel


In [72]:
rdt = RDTModel()

[2020-12-30 10:27:12,763] [INFO] loading projection weights from /home/artkra/SOURCES/faqer/models/rdt/all.norm-sz100-w10-cb0-it1-min100.w2v
[2020-12-30 10:27:19,362] [WARNING] duplicate word '……………………………………………………………………………………' in word2vec file, ignoring all but first
[2020-12-30 10:27:19,539] [INFO] duplicate words detected, shrinking matrix size from 1239965 to 1239964
[2020-12-30 10:27:19,540] [INFO] loaded (1239964, 100) matrix from /home/artkra/SOURCES/faqer/models/rdt/all.norm-sz100-w10-cb0-it1-min100.w2v
[2020-12-30 10:27:19,540] [INFO] precomputing L2-norms of word weight vectors


In [74]:
from collections import Counter

words = []
for line in lines:
    for word in line:
        if word in rdt.w2v.vocab:
            words.append(word)

In [79]:
c = Counter(words)
keys = sorted(c, key=lambda x: c[x], reverse=True)

In [86]:
tokens = lines[0]
trigrams = [([tokens[i], tokens[i + 1], tokens[i + 2]]) for i in range(len(tokens) - 2)]

In [87]:
trigrams

[['would', 'like', 'get'],
 ['like', 'get', 'email'],
 ['get', 'email', 'password'],
 ['email', 'password', 'reset'],
 ['password', 'reset', 'mailto'],
 ['reset', 'mailto', 'award']]